In [105]:
import swat
import os
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import lightgbm as lgb
import time
import gc
from sklearn.model_selection import train_test_split
from lightgbm import early_stopping
try:
    import cPickle as pickle
except BaseException:
    import pickle
import time

import sasctl.pzmm as pzmm
from sasctl import Session
from sasctl.services import model_repository as modelRepo
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from pathlib import Path
from sasctl.services import model_repository as mr

In [106]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = "C:/Users/turkak/SASVIYACERT/my_ca_certificate.pem"
hostname = 'https://server.demo.sas.com:443/cas-shared-default-http/'
username = 'sasdemo'
password = 'Orion123'
conn = swat.CAS('https://server.demo.sas.com:443/cas-shared-default-http/', username='sasdemo',password='Orion123')

In [107]:
conn.serverStatus()

NOTE: Grid node action status report: 1 nodes, 9 total actions executed.


[About]

 {'CAS': 'Cloud Analytic Services',
  'CASCacheLocation': 'CAS Disk Cache',
  'CASHostAccountRequired': 'OPTIONAL',
  'Copyright': 'Copyright © 2014-2022 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2022-08-07T05:40:26Z',
  'System': {'Hostname': 'controller.sas-cas-server-default.sas-viya.svc.cluster.local',
   'Linux Distribution': 'Red Hat Enterprise Linux release 8.6 (Ootpa)',
   'Model Number': 'x86_64',
   'OS Family': 'LIN X64',
   'OS Name': 'Linux',
   'OS Release': '3.10.0-1160.6.1.el7.x86_64',
   'OS Version': '#1 SMP Tue Nov 17 13:59:11 UTC 2020'},
  'Transferred': 'NO',
  'Version': '4.00',
  'VersionLong': 'V.04.00M0P07182022',
  'Viya Release': '20220721.1658437616927',
  'Viya Version': 'Stable 2022.1.3',
  'license': {'expires': '22Oct2022:00:00:00',
   'gracePeriod': 45,
   'site': 'VIYA 4 W ESP, ACCESS, VA, VDMML, AIOT, AND PQA',
   'siteNum': 70180938,
   'warningPeriod': 49}}

[nodestatus]

 Node Status
 
                                                 name        role  uptime  running  stalled
 0  controller.sas-cas-server-default.sas-viya.svc...  controller   4.752        0        0

[server]

 Server Status
 
    nodes  actions
 0      1        9

+ Elapsed: 0.00075s, user: 0.000559s, sys: 0.000163s, mem: 0.322mb

In [108]:
if not os.path.exists('Models'):
    os.makedirs('Models')

In [ ]:
#tbl = conn.read_csv("./datasets/hmeq.csv",casout=dict(name='hmeq'))


In [ ]:

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

#  Modelling Using SAS Cluster

In [109]:
#Load Required Actionset
Actionsetlist = ["sampling","FedSQL","lightgradboost"]
for act in range(len(Actionsetlist)):
    conn.loadactionset(Actionsetlist[act])

NOTE: Added action set 'sampling'.
NOTE: Added action set 'FedSQL'.
NOTE: Added action set 'lightgradboost'.


In [123]:
RawTable = 'HMEQ'
RawTablelib = 'casuser'
Target = "BAD"
OversamplingOutTable = RawTable + '_Out_Over'
PartitionedTable = RawTable + '_Part'

In [111]:
#Connect Data and assign
def AssignRawTable(table,library):
    tbl = conn.CASTable(name=RawTable,caslib=RawTablelib)
    return tbl
    

In [112]:
tbl = AssignRawTable(table=RawTable,library=RawTablelib) 

In [11]:
#tbl.fetch(to=10)

In [12]:
res = conn.simple.freq(                           # 1
    inputs = [{"name":"bad"}],                # 2
    table = {"name":"hmeq"},                   # 3
    casOut = {"name":"hmeq_freq",              # 4
              "replace":True})

res = conn.table.fetch(table = "hmeq_freq")       # 5
from IPython.core.display import display, HTML
display(res["Fetch"])

piedata = conn.CASTable(                          # 6
    "hmeq_freq").to_frame().sort_values(['_Frequency_'],
                                        ascending=False)



C:\Users\turkak\AppData\Local\Temp\ipykernel_15256\1169668044.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,_Column_,_Numvar_,_Fmtvar_,_Level_,_Frequency_
0,BAD,1.0,1,1.0,1189.0
1,BAD,0.0,0,2.0,4771.0


In [ ]:
'''import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (4.8, 3.6))
plt.title("Models Produced by Vehicle Type")
ax.pie(piedata._Frequency_, labels = piedata._Fmtvar_, 
       radius=1.1, autopct = "%.1f%%", pctdistance=0.75)
ax.get_figure()
'''

In [60]:
#oversdict = dict(partind =True,eventprop = 0.3,event ="1",samppctevt=90)

In [128]:
def oversample(Table=RawTable,OverSamplingOut=OversamplingOutTable,**kwargs):
    
    DisplayDict = dict(names = "OVERFreq") 
    OverSamplingOutDict = dict(casout=dict(name=OverSamplingOut,replace=True),copyVars= "ALL")
    OverSamplingTableDict = dict(name = RawTable,groupBy = [Target]) # Oversampling works with groupBy Column -Target Field 
    OutputTablesDict = dict(names = "OVERFreq",replace = True)
    
    conn.oversample(display=DisplayDict,
                    output=OverSamplingOutDict,
                    partind=kwargs['partind'],
                    seed=10,
                    eventprop=kwargs['eventprop'],
                    event=kwargs['event'],
                    samppctevt=kwargs['samppctevt'],
                    table=OverSamplingTableDict,
                    outputTables=OutputTablesDict)
    
    over_out=conn.CASTable(OversamplingOutTable)
    return over_out

In [130]:
over_out = oversample(Table=RawTable,OverSamplingOut=OversamplingOutTable,partind=False,eventprop=0.3,event ="1",samppctevt=90)

NOTE: Oversampling is in effect.
NOTE: Using SEED=10 for sampling.
NOTE: The CAS table 'OVERFreq' in caslib 'CASUSER(sasdemo)' has 2 rows and 4 columns.


In [122]:
conn.CASTable('OVERFreq').fetch(to=10)

,ByGrpID,BAD,NObs,NSamp
0,0,0,4771,2497
1,1,1,1189,1070


In [ ]:
'''
# Defining Oversample Parameters
DisplayDict = dict(names = "OVERFreq") 
OverSamplingOutDict = dict(casout=dict(name="hmeq_out_over",replace=True),copyVars= "ALL")
OverSamplingTableDict = dict(name = "hmeq",groupBy = ["bad"]) # Oversampling works with groupBy Column -Target Field 
OutputTablesDict = dict(names = "OVERFreq",replace = True)
'''

In [ ]:
'''
conn.oversample(display=DisplayDict,
             output=OverSamplingOutDict,
             partind=False, seed=10, eventprop=0.3, event="1", samppctevt=90,
             table=OverSamplingTableDict,
             outputTables=OutputTablesDict)

#over_out=conn.CASTable('hmeq_out_over')
#over_out.fetch(to=20)

#over_out2=conn.CASTable('OVERFreq')
#over_out2.fetch()
'''

In [196]:
# Partitioning the Data
def createpartition(samppct):
    PartitionedTable = RawTable + '_Part'
    hmeq_part = conn.CASTable(PartitionedTable, replace=True
                              ,computedVarsProgram = 'BAD_C = put(BAD,1.);' # (Create a new variable with program)
                             )
    
    over_out.groupby(Target).sampling.stratified(output=dict(casout=hmeq_part, copyvars='all'),
                                                 samppct=samppct,
                                                 partind=True
                                                )
    
    return hmeq_part

In [197]:
hmeq_part = createpartition(70)

NOTE: Stratified sampling is in effect.
NOTE: Using SEED=116373010 for sampling.


In [ ]:
#stra_out.fetch(to=20)

In [ ]:
#stra_out2=conn.CASTable('STRAFreq')
#stra_out2.fetch(to=20)



In [ ]:
#stra_out3=conn.CASTable('PartIndMap')
#stra_out3.fetch()

In [201]:
Train = hmeq_part.query('_partind_ = 1')
Test = hmeq_part.query('_partind_ = 0')

In [ ]:
hmeq_part_1.dtypes

In [ ]:
nominals = []
cattypes = ['varchar','char', 'categorical', 'object']
for cat in cattypes:
    nominals.append(cat)
hmeq_part_1.dtypes

In [214]:
feature_list = Train.columns.to_list()
vartypes = Train.dtypes
drop_cols = ['_Freq_','_PartInd_',Target,'BAD_C']
feature_list = [x for x in feature_list if (x not in drop_cols )]

In [216]:

numeric_list = [x for x in feature_list if (vartypes == 'double')[x] == True]
nominal_list = [x for x in Model_Inputs if (vartypes == 'double')[x] == False]
Model_Inputs = numeric_list + nominal_list

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,_Freq_,_PartInd_,BAD_C
0,1.0,1100.0,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN,0.664989,1.0,1
1,1.0,1500.0,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN,0.664989,1.0,1
2,1.0,1500.0,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.664989,1.0,1
3,1.0,1700.0,30548.0,40320.0,HomeImp,Other,9.0,0.0,0.0,101.466002,1.0,8.0,37.113614,0.664989,1.0,1
4,1.0,1800.0,48649.0,57037.0,HomeImp,Other,5.0,3.0,2.0,77.100000,1.0,17.0,NaN,0.664989,1.0,1


In [235]:
def lgb_sas():
    numeric_list = [x for x in feature_list if (vartypes == 'double')[x] == True]
    nominal_list = [x for x in feature_list if (vartypes != 'double')[x] == True]
    Model_Inputs = numeric_list + nominal_list
    conn.lightgradboost.lgbmTrain(inputs=numeric_list,objective="BINARY",
        nominals=[Target],
        #nominals=nominal_list.append(Target),                      
        display= {'names':['IterHistory','ModelInfo','NObs']}  ,                
        output={'casout':{'name':'scored_train','replace':True}},
        savestate={'name':'savedstate','replace':True},
        seed=12345,
        table=Train,
        target=Target,
        validtable=Test,
        #outputTables={'names':['IterHistory_Train','ModelInfo','NObs'],'replace':True})
        outputTables = {"includeAll":True,"replace":True})

In [236]:
lgb_sas()

,RowId,Description,cValue,Value
0,BOOSTING,boosting method,gbdt,NaN
1,OBJECTIVE,objective type,binary,NaN
2,NBINS,Number of Bins,,255.0
3,LEAFSIZE,Minimum Leaf Size,,20.0
4,LASSO,Lasso (L1) penalty,,0.0
5,RIDGE,Ridge (L2) penalty,,0.0
6,MAXCAT,Maximal Categories,,32.0


In [221]:
conn.lightgradboost.lgbmTrain(inputs=numeric_list,objective="BINARY",
        #nominals=['REASON','JOB','BAD'],
        nominals=[Target],                      
        display= {'names':['IterHistory','ModelInfo','NObs']}  ,                
        output={'casout':{'name':'scored_train','replace':True}},
        savestate={'name':'savedstate','replace':True},
        seed=12345,
        table=hmeq_part_1,
        target='BAD',
        validtable=hmeq_part_0,
        #outputTables={'names':['IterHistory_Train','ModelInfo','NObs'],'replace':True})
        outputTables = {"includeAll":True,"replace":True})
lgb_out =conn.CASTable('scored_train')

NOTE: The misclassification error is     0.00.
NOTE: 345489 bytes were written to the table "savedstate" in the caslib "CASUSER(sasdemo)".
NOTE: The CAS table 'ModelInfo' in caslib 'CASUSER(sasdemo)' has 7 rows and 4 columns.
NOTE: The CAS table 'NObs' in caslib 'CASUSER(sasdemo)' has 4 rows and 3 columns.
NOTE: The CAS table 'IterHistory' in caslib 'CASUSER(sasdemo)' has 100 rows and 3 columns.
NOTE: lgbmTrain completed successfully.


In [238]:
conn.loadactionset('aStore')

m=conn.describe(
   rstore='savedstate',
   epcode=True
)
print(m.Description)
print(m.InputVariables)
print(m.OutputVariables)
print(m.epcode)

NOTE: Added action set 'aStore'.
Basic Information

         Attribute               Value
0  Analytic Engine      lightGradBoost
1     Time Created  07Aug2022:08:17:48
Input Variables

      Name  Length Role Type RawType FormatName
0     LOAN     8.0   Id          Num           
1  MORTDUE     8.0   Id          Num           
2    VALUE     8.0   Id          Num           
3      YOJ     8.0   Id          Num           
4    DEROG     8.0   Id          Num           
5   DELINQ     8.0   Id          Num           
6    CLAGE     8.0   Id          Num           
7     NINQ     8.0   Id          Num           
8     CLNO     8.0   Id          Num           
9  DEBTINC     8.0   Id          Num           
Output Variables

     Name  Length       Type             Label
0   I_BAD    16.0  Character         Into: BAD
1  P_BAD0     8.0        Num  Predicted: BAD=0
2  P_BAD1     8.0        Num  Predicted: BAD=1
data sasep.out;
  dcl package score sc();
  dcl double "LOAN";
  dcl double "MOR

In [ ]:
conn.CASTable('IterHistory').fetch(from_ =80 ,to=100)

In [ ]:

conn.score(
   table=hmeq_part_0,
   casout={'name':'lgb_score_out','replace':True},
   rstore='savedstate'
)
m=conn.fetch(table={'name':'lgb_score_out'})
print(m)

In [ ]:
m

In [ ]:
lgb_out['I_BAD'].count()

In [ ]:
hmeq_part_0['BAD'].count()

In [ ]:
#conn.close()